In [1]:
CONFIG_PATH = "configs/small_most_votes_rand_init_test_diff.yml"

In [2]:
from utils import get_config, join_sets, load_model, print_log, save_model, load_model_dict
from process_data import get_model_input
from model import get_best_model
from superdebug import debug
config = get_config(CONFIG_PATH, "_manipulation", print_config = False)
config["batch_size"] = 1
target, original_feature_map, categorical_features, string_features, train_data, test_data, test_data_info, train_submission_upvote_df, num_all_users = get_model_input(config)
model, token_embedding = get_best_model(config, categorical_features, string_features, original_feature_map)

为了使用邮件提醒功能，请设置环境变量MY_QQ_EMAIL（QQ邮箱地址）与MY_QQ_EMAIL_PWD（QQ邮箱授权码）
------------------ 2022-08-01 08:34:23 ------------------
GPU ready...
Smart using cuda:1
------------------ 2022-08-01 08:34:25 ------------------
DEBUG: at /home/TableSense/largedisk/wanrong/Curation-Modeling/process_data.py:327 get_model_input
Loading prepared data...
------------------ 2022-08-01 08:34:25 ------------------
------------------ 2022-08-01 08:34:31 ------------------
DEBUG: at /home/TableSense/largedisk/wanrong/Curation-Modeling/model.py:239 get_tokenizer
Initializing the tokenizer from sbert-base-uncased
------------------ 2022-08-01 08:34:31 ------------------
------------------ 2022-08-01 08:34:32 ------------------
DEBUG: 1 vars: ['original_token_num'], at /home/TableSense/largedisk/wanrong/Curation-Modeling/model.py:242 get_tokenizer
0 / 1.  original_token_num num val: 30522
------------------ 2022-08-01 08:34:32 ------------------
------------------ 2022-08-01 08:34:32 ------------------
DEBUG: 1 var

In [3]:
import numpy as np
from train import get_data_loader, pad_sequences
model = model.eval()

testset, test_loader = get_data_loader(config, test_data, model.tokenizer, categorical_features, string_features, target, interactive = True, add_target_user_ratio = 0, shuffle=False, batch_size=1)


In [4]:
from matplotlib import interactive
import torch
from tqdm import tqdm
from train import to_device
from dynamic_data import CollateFN
collate_fn = CollateFN(model.tokenizer)
# print("Total user num:", user_embedding.shape[0])
model.to(model.device)
for i in range(len(testset)):
    while True:
        test_input = testset[i]
        if test_input is None:
            break
        else:
            input_string, label, weight, df_index = test_input
        input_ids, token_type_ids, attention_mask, label, weight, df_index = collate_fn([[input_string, label, weight, df_index]])
        user = model.tokenizer.convert_ids_to_tokens([int(input_ids[0, 2])])[0]
        print("User:", user, "Ground truth:", int(label))
        input_ids, token_type_ids, attention_mask, label, weight = to_device(model.device, False, input_ids, token_type_ids, attention_mask, label.float(), weight)
        y_pred = model(input_ids, token_type_ids, attention_mask).cpu().data.numpy()  # .squeeze()
        print(y_pred[0][0], "Upvote" if y_pred[0][0] > 0.5 else "Downvote")
        print("---------------------")

Original upvoted users: []
New upvoted users: [1]
Original downvoted users: []
New downvoted users: []
User: USERNAME_7526 Ground truth: 1
0.9853952 Upvote
---------------------
Original upvoted users: []
New upvoted users: []
Original downvoted users: []
New downvoted users: []
User: USERNAME_7526 Ground truth: 1
0.985797 Upvote
---------------------
Original upvoted users: []
New upvoted users: [7526]
Original downvoted users: []
New downvoted users: []
User: USERNAME_7526 Ground truth: 1
0.9853952 Upvote
---------------------
Original upvoted users: []
New upvoted users: []
Original downvoted users: []
New downvoted users: [7526]
User: USERNAME_7526 Ground truth: 1
0.9853792 Upvote
---------------------
Original upvoted users: []
New upvoted users: []
Original downvoted users: []


ValueError: invalid literal for int() with base 10: 'a'